# LDA Workshop


Blog Post นี้เขียนไว้ดีมาก ถ้าอยากจะทบทวนวิธีการใช้อ่านผลจาก LDA 

https://neptune.ai/blog/pyldavis-topic-modelling-exploration-tool-that-every-nlp-data-scientist-should-know

# Tweet เกี่ยวกับอะไรบ้าง

In [19]:
import pandas as pd
data_tokenized = pd.read_csv('tweet_tokenized.csv')
docs = [eval(x) for x in data_tokenized.tok.to_list()[:-1]]

In [20]:
!pip install gensim

In [21]:
!pip install pythainlp

In [22]:
from gensim.models.ldamodel import LdaModel
from gensim import corpora
from collections import Counter
from pythainlp.corpus import thai_stopwords

In [23]:
import re
patt = re.compile('^[ก-์a-zA-Z]+$')
new_doc_list = []
all_words = []
for doc in docs:
    new_doc = []
    for w in doc:
        if w not in thai_stopwords() and patt.match(w):
            new_doc.append(w)
            all_words.append(w)
    new_doc_list.append(new_doc)
    

In [24]:
c = Counter(all_words)

In [25]:
c.most_common(20)

[('RT', 11201),
 ('https', 4353),
 ('คน', 2726),
 ('ทหาร', 2308),
 ('ทำ', 2097),
 ('ดี', 1695),
 ('ประชาชน', 1619),
 ('กองทัพบก', 1375),
 ('ไทย', 1222),
 ('กำลังใจ', 1057),
 ('พรรค', 946),
 ('เรื่อง', 923),
 ('WassanaNanuam', 916),
 ('อนาคต', 867),
 ('งาน', 857),
 ('ข่าว', 833),
 ('ตัว', 804),
 ('น้ำ', 719),
 ('ชีวิต', 705),
 ('ไฟ', 669)]

# Train LDA 
Assume ว่าข้อมูลอยู่ในรูป list of list of words แล้ว

In [26]:
import re
patt = re.compile('^[ก-์a-zA-Z]+$')
custom_stop = {'RT', 'https'}
doc_list_lda = []
for doc in docs:
    new_doc = []
    for w in doc:
        if w not in thai_stopwords() and patt.match(w) and w not in custom_stop:
            new_doc.append(w)
            all_words.append(w)
    doc_list_lda.append(new_doc)
    
lda_dict = corpora.Dictionary(doc_list_lda)
data_set = []
for doc in doc_list_lda:
    data_set.append(lda_dict.doc2bow(doc))
model = LdaModel(corpus=data_set, num_topics=10, id2word=lda_dict, 
                 alpha='auto', eval_every=10, random_state=10)

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt

# Topic-keys

In [27]:
model.show_topic(0)

[('ดี', 0.06718646),
 ('ยอด', 0.030424874),
 ('ทำ', 0.018884268),
 ('เยี่ยม', 0.017669631),
 ('ท่าน', 0.016135111),
 ('Area', 0.014205389),
 ('ชอบ', 0.01255456),
 ('WassanaNanuam', 0.01226222),
 ('JuahuaHeadline', 0.01146437),
 ('สร้าง', 0.0110531375)]

In [28]:
for i in range(model.num_topics):
    print (str(i) + ':' + ' '.join([w[0] for w in model.show_topic(i)]))

0:ดี ยอด ทำ เยี่ยม ท่าน Area ชอบ WassanaNanuam JuahuaHeadline สร้าง
1:สู้ WassanaNanuam พระเจริญ weloverta ดู โดน ลุง ด่า กู้ ตัว
2:กำลังใจ คน Army ทำ Area ดี เจ้าหน้าที่ ไวรัส ป้องกัน ท่าน
3:ไฟ ป่า ดับ กอง ยิง น้ำ หน่วย กราด เตรียม ประชาชน
4:กองทัพบก ทำ ฝึก ไทย แนว วิ่ง เพื่อประชาชน กิจกรรม จิตอาสา แดน
5:คน น้อง ทหาร จุด ป่า ทีม ตรวจ ผม เหตุการณ์ สมอง
6:พรรค อนาคต เรื่อง ยุบ คน FWP อย่า ทำ ข่าว Piyabutr
7:เสียใจ ชีวิต ครอบครัว บิน บก พระราชทาน หน่วย TV งาน บัญชา
8:ทหาร ประชาชน กองทัพบก หน้ากาก ช่วยเหลือ อนามัย ที่พึ่ง โอกาส กล้า news
9:ภัย VoiceTVOfficial ข่าว น้ำ พื้นที่ กองทัพบก ปลอด ทหาร ช่วยเหลือ ประชาชน


In [29]:
!pip install pyldavis

In [30]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

pyLDAvis.enable_notebook()

In [31]:
vis = gensimvis.prepare(model, data_set, lda_dict)
vis

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.031773 -0.183528       1        1  14.215098
6      0.190490  0.103116       2        1  12.135062
9     -0.219367  0.015313       3        1  10.618979
0      0.129404 -0.101189       4        1   9.707577
8     -0.175235 -0.026470       5        1   9.609545
7     -0.062022  0.061264       6        1   9.559464
3     -0.097739  0.188021       7        1   9.260243
5     -0.017540  0.015285       8        1   8.490872
1      0.202898  0.116910       9        1   8.427897
4      0.017338 -0.188721      10        1   7.975263, topic_info=         Term         Freq        Total Category  logprob  loglift
31       ทหาร  2461.000000  2461.000000  Default  30.0000  30.0000
2771     Army  1144.000000  1144.000000  Default  29.0000  29.0000
70         ดี  1851.000000  1851.000000  Default  28.0000  28.0000
47    กำลังใจ  1558.000000  1558.000000  Default  27.0000  27.0000
79    ประชาชน  1920.000000  1920.000000  Default  26.0000  26.0000
...       ...          ...          ...      ...      ...      ...
1289      สาย    96.083505   230.252649  Topic10  -4.9514   1.6549
155    กฎหมาย    81.924715   165.174572  Topic10  -5.1108   1.8276
427     เมือง    89.823585   273.241125  Topic10  -5.0188   1.4163
47    กำลังใจ   119.784039  1558.144695  Topic10  -4.7309  -0.0367
70         ดี    97.147786  1851.108978  Topic10  -4.9404  -0.4185

[649 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
2459      7  0.985420  ARMdhiravath
2412      4  0.956695     Alitsarah
3887      1  0.754719          Area
3887      2  0.003068          Area
3887      4  0.240324          Area
...     ...       ...           ...
225       2  0.112622           ไอ้
225       4  0.183751           ไอ้
225       7  0.082984           ไอ้
225       8  0.011855           ไอ้
225       9  0.491979           ไอ้

[1502 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 7, 10, 1, 9, 8, 4, 6, 2, 5])

# Doc-topic 

In [ ]:
doc_topics = model.get_document_topics(data_set)

In [ ]:
def print_doc_for_topic(doc_topics, doc_list_lda, topic_no):
    printed = set()
    for i in range(len(doc_list_lda)):
        top_topic, top_proportion = doc_topics[i][0]
        if top_topic == topic_no and top_proportion > 0.5:
            to_print = ''.join(doc_list_lda[i])
            if to_print not in printed:
                print (to_print)
                print ('------')
            printed.add(to_print)
            #doc_topics[0]
print_doc_for_topic(doc_topics, docs, 2)

🇹🇭ประเทศไทยจะเข้าสู่การระบาดมากหรือน้อย ขึ้นอยู่กับ “พลังอำนาจของคนไทยทุกคน “  ถ้าเริ่มมีอาการเป็นไข้ พักอยู่บ้าน รักษาตามอาการ  ถ้าเริ่มมีการหนักมากขึ้น  ต้องมาพบแพทย์ ต้องเตรียมการป้องกันกันติดต่อ เช่น  สวมหน้ากาก เตรียมเจลล้างมือ  ลดการแพร่ไปสู่คนอื่น.... https://t.co/6MfZQz2Z2b
------
การฝึกแลกเปลี่ยนความรู้ ระหว่าง ทหารไทย และ ทหาร สหรัฐอเมริกา Cross Training Exercise (CTX) สถานีที่ ๒ ดำรงชีพในป่า ณ สนามฝึกยุทธวิธี ทภ.๓ บ้านด่านลานหอย จ.สุโขทัย...

https://t.co/npbg8b2AvZ https://t.co/bmRUfysrMg
------
“มาดามเดียร์” สอน “ช่อ” ไม่รู้ให้ถาม กรณีขายหุ้นสื่อมวลชน มีหลักฐานครบถ้วน https://t.co/ePpEjsnY2P ผ่าน @PPTVHD36
------
RT @MaisonWanvipa: #แรงงานน่ารู้

.
กรณีที่มีการปิดโรงงานออกข่าวเยอะมาก ตั้งแต่ปี 62 จนถึงปัจจุบัน ทำให้พี่น้องเราเดือดร้อนหลายพันคน อีกกร…
------
ผมไม่ใช่ดารา จะติดตามชีวิตผมไปถึงไหนครับเอาเวลาเผือกไปทำอย่างอื่นเถอะเผื่อชีวิตจะดีขึ้นนนนนนนน🧐
------
"บิ๊กตู่” ย้ำ ทำงานเต็มที่ให้ชายแดนใต้สงบ-ปลอดภัย แนะปชช.ร่วมมือแก้ปัญหาได้ โชว์พูดยาวีกับเด็กปัตตานี | สยามรัฐ http

# Exercise 1
ลองเปลี่ยนจำนวน topic แล้วสังเกตุดูว่าผลเปลี่ยนไปบ้างมั้ย ยังได้ข้อสรุปเดิมอีกหรือไม่

# Exercise 2

Train topic model จากไฟล์ `prachathai-news-sample-2000.txt` ซึ่งอยู่ในรูป one line per doc 
ก่อนอื่นเราจะต้องเปลี่ยนข้อมูลให้อยู่ในรูป list of (list of words) ก่อนและทำการ pre-processing โดยการ
1. word tokenize
2. เอา stopwords ออกไปก่อนให้เรียบร้อย 
3. เก็บเอาเฉพาะคำที่เป็นภาษาไทย ไม่เอาตัวเลข ไม่เอาภาษาอังกฤษ

ให้ copy code ข้างบนเพื่อเทรน LDA และวิเคราะห์ผลจาก topic keys ที่ได้ อาจจะต้องมีการปรับจำนวน topic ให้เหมาะสม